In [ ]:
# import das libs necessárias
import pandas as pd # trabalhar com dataframes
import numpy as np # realizacao de algumas operacoes com matrizes

#imagens
import cv2 # transformacoes faceis em imagens
from PIL import Image # trabalhar com imagens

# ferramentas
import glob # exploracao de diretorios
from pylab import *
import tqdm
import importlib
import time

# plot 
import matplotlib.pyplot as plt # plotagem
%matplotlib inline

# Machine Learning
from sklearn import svm
from sklearn.model_selection import GridSearchCV # Grid Search para rodar todos os parametros

#importando ferramentas já implementadas anteriormente
import sampling #retorna uma amostra para treino
importlib.reload(sampling)
import PCA_and_Inverse
importlib.reload(PCA_and_Inverse)
import feature_selection
importlib.reload(feature_selection)

## Cria uma amostra das imagens do tamanho definido e carrega em dataframes

In [ ]:
#carregando amostra dos dados
X, Y = sampling.get_sample(400)

## Extração de características

- Calcula as Componentes principais para os dados carregados

In [ ]:
pca = PCA_and_Inverse.PCA_and_Inverse()
pca_components = pca.get_PCA(X=X, n_components=64)

## Seleção de características

- Calcula as melhores features a serem utilizadas pelos métodos de:
    - Informação Mútua
    - Algoritmo Genético

#### Informação Mútua

In [ ]:
mic_threshold = 0.05
mic_features_df = feature_selection.get_MIC_features(X=X, Y=Y)
mic_features = list(mic_features_df[mic_features_df[0] > mic_threshold].index)

#### Algoritmo Genético

In [ ]:
ga_features = feature_selection.get_GA_features(X=X, Y=Y)
ga_features

## Gera as combinações de parâmetros para rodar o SVM

In [ ]:
svm_classifier = svm.SVC()
parameters = [{"kernel":["linear"], "C":[0.1, 1, 10]},
              {"kernel":["rbf"], "C":[0.1, 1, 10], "gamma":[0.1, 1, 10]},
              {"kernel":["sigmoid"], "C":[0.1, 5, 10], "coef0":[1,5], "gamma":[0.1, 1, 10]},
              {"kernel":["poly"], "C":[0.1, 5, 10], "coef0":[1,5], "degree":[1, 2, 3], "gamma":[0.1, 1, 10]}
             ]

#### Roda todas as combinações da SVM para as features

In [ ]:
grid_search_full = GridSearchCV(svm_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_full.fit(X, Y)

In [ ]:
grid_search_full.cv_results_

#### Roda todas as combinações da SVM para as features extraídas pelo PCA

In [ ]:
grid_search_pca = GridSearchCV(svm_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_pca.fit(pca_components, Y)

In [ ]:
grid_search_pca.cv_results_

#### Roda todas as combinações da SVM para as features selecionadas pela Informação Mútua (MIC)

In [ ]:
grid_search_mic = GridSearchCV(svm_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_mic.fit(X[mic_features], Y)

In [ ]:
grid_search_mic.cv_results_

#### Roda todas as combinações da SVM para as features selecionadas pelo Algoritmo Genético (GA)

In [ ]:
grid_search_ga = GridSearchCV(svm_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_ga.fit(pca_components, Y)

In [ ]:
grid_search_ga.cv_results_